In [2]:
import sys
import json
import os
os.environ['CUDA_VISIBLE_DEVICES']='4'

import fire
# import gradio as gr
import torch
import transformers
from peft import PeftModel
from transformers import GenerationConfig, LlamaForCausalLM, LlamaTokenizer, AutoTokenizer, AutoModelForCausalLM, AutoModel, TextStreamer

# from utils.prompter import Prompter
import bitsandbytes as bnb
import warnings
warnings.filterwarnings('ignore')


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /data/Yucong/anaconda3/lib/python3.9/site-packages/bitsandbytes/libbitsandbytes_cuda113.so
CUDA SETUP: CUDA runtime path found: /data/Yucong/anaconda3/lib/libcudart.so.11.0
CUDA SETUP: Highest compute capability among GPUs detected: 8.0
CUDA SETUP: Detected CUDA version 113
CUDA SETUP: Loading binary /data/Yucong/anaconda3/lib/python3.9/site-packages/bitsandbytes/libbitsandbytes_cuda113.so...


/data/Yucong/anaconda3/lib/python3.9/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: Found duplicate ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] files: {PosixPath('/data/Yucong/anaconda3/lib/libcudart.so.11.0'), PosixPath('/data/Yucong/anaconda3/lib/libcudart.so')}.. We'll flip a coin and try one of these, in order to fail forward.
Either way, this might cause trouble in the future:
If you get `CUDA error: invalid device function` errors, the above might be the cause and the solution is to make sure only one ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] in the paths that we search based on your env.
  warn(msg)
2023-10-03 17:25:45.406748: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0


In [3]:
print(torch.cuda.is_available())

True


In [4]:
load_8bit = False
model_name = 'ChatGLM2'

if model_name == 'LLaMA2':
    base_model = 'meta-llama/Llama-2-7b-chat-hf'
    model_path = 'meta-llama/Llama-2-7b-chat-hf'
elif model_name == 'ChatGLM2':
    base_model = 'THUDM/chatglm2-6b'
    model_path = 'THUDM/chatglm2-6b'

if torch.cuda.is_available():
    device = "cuda"

In [5]:
if model_name in ['ChatGLM2']:
    from transformers import AutoTokenizer, AutoModel
    tokenizer = AutoTokenizer.from_pretrained(base_model, trust_remote_code=True)
    model = AutoModel.from_pretrained(base_model, trust_remote_code=True).half().cuda()
    model = model.eval()


if torch.__version__ >= "2" and sys.platform != "win32":
    model = torch.compile(model)

Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

In [6]:
import json
import os.path as osp
from typing import Union

def evaluate(
    instruction,
):
    response, history = model.chat(tokenizer, instruction, history=[])
    
    return instruction, response

In [8]:
instruction = 'hello'
instruction, response = evaluate(instruction)
print(response)

Hello! How can I assist you today?


In [38]:
import pandas as pd
from recbole.quick_start import load_data_and_model
dataset_name = 'steam'
# rec_model_path = '../RecBole/experiments/steam/BPR/BPR-Aug-28-2023_11-58-38.pth'
# rec_model_path = '../RecBole/experiments/steam/SASRec/SASRec-Aug-28-2023_11-58-33.pth'
rec_model_path = '../RecBole/experiments/steam/LightGCN/LightGCN-Aug-28-2023_12-02-29.pth'

config, rec_model, dataset, train_data, valid_data, test_data = load_data_and_model(
    model_file=rec_model_path,
)

df_item = pd.read_csv(f'dataset/{dataset_name}/{dataset_name}.item', sep='\t')
df_inter = pd.read_csv(f'dataset/{dataset_name}/{dataset_name}.inter', sep='\t')
if 'ml' in dataset_name:
    df_user = pd.read_csv(f'dataset/{dataset_name}/{dataset_name}.user', sep='\t')

03 Oct 18:02    INFO  
General Hyper Parameters:
gpu_id = 7
use_gpu = True
seed = 2020
state = INFO
reproducibility = True
data_path = dataset/steam
checkpoint_dir = experiments/steam/LightGCN
show_progress = True
save_dataset = False
dataset_save_path = None
save_dataloaders = False
dataloaders_save_path = None
log_wandb = True

Training Hyper Parameters:
epochs = 500
train_batch_size = 4096
learner = adam
learning_rate = 0.0005
train_neg_sample_args = {'distribution': 'uniform', 'sample_num': 1, 'alpha': 1.0, 'dynamic': False, 'candidate_num': 0}
eval_step = 1
stopping_step = 10
clip_grad_norm = None
weight_decay = 0.0
loss_decimal_place = 4

Evaluation Hyper Parameters:
eval_args = {'group_by': 'user', 'order': 'TO', 'split': {'LS': 'valid_and_test'}, 'mode': 'full'}
repeatable = False
metrics = ['NDCG', 'Recall', 'MRR']
topk = [10, 20]
valid_metric = NDCG@10
valid_metric_bigger = True
eval_batch_size = 4096
metric_decimal_place = 4

Dataset Hyper Parameters:
field_separator = 	
seq

In [39]:
from recbole.utils.case_study import full_sort_topk


def get_instruction_steam(idx):
    # history_inter_info
    df_inter_user = df_inter[df_inter['user_id:token'] == int(idx)]
    his_id_list = df_inter_user['product_id:token'].tolist()[-10:]
    uid_series = dataset.token2id(dataset.uid_field, [idx])
    topk_score, topk_iid_list = full_sort_topk(uid_series, rec_model, test_data, k=1, device=config['device'])
    rec_list = dataset.id2token(dataset.iid_field, topk_iid_list.cpu()) 
    # next_item_info
    next_item = df_item[df_item["id:token"] == int(rec_list)]
    next_item_title = next_item["app_name:token"].values[0]
    next_item_tag = next_item["genres:token_seq"].values[0]
    next_item_price = next_item["price:float"].values[0]

    # print(f'The price of the game is {next_item_price}.')
    instruction = 'The history games played by the customer are:\n'
    for i, id in enumerate(his_id_list):
        item = df_item[df_item["id:token"] == id]
        item_title = item["app_name:token"].values[0]
        item_tag = item["genres:token_seq"].values[0]
        item_price = item["price:float"].values[0]


        instruction += f'{i+1}: {item_title}' + '\n'
        instruction += f'The tags of the game are {item_tag}.\n'
        instruction += f'The price of the game is {item_price}.\n'

    instruction += f"""
The recommender system suggests the customer to play this game with the following title and tags:
{next_item_title}.
The tags of the game are {next_item_tag}.

Your mission is to infer the game's information from the history record, such as price. \
You must infer price. Do not return other information. And DO NOT return Unknow or Null.
DO NOT RETURN OTHER INFORMATION!!! Only return the JSON format below!!!! So that the system can evaluate your answer.


The output format is using JSON format as follows:
{{
    "Game price": "<Free to Play or only number of the price>"
}}
"""
    return instruction

if dataset_name == 'steam':
    get_instruction = get_instruction_steam

In [33]:
from tqdm import tqdm
def get_valid_user_list(df):
    valid_user_list = []
    for id in tqdm(df['user_id:token'].unique().tolist()):
        if len(df_inter[df_inter['user_id:token'] == id]) >=5:
            valid_user_list.append(id)
        if len(valid_user_list) == 1000:
            break
    return valid_user_list
valid_user_list = get_valid_user_list(df_inter)
valid_user_list

  0%|          | 9549/2567538 [00:25<1:55:59, 367.58it/s]


[13,
 17,
 31,
 41,
 42,
 75,
 78,
 80,
 85,
 94,
 98,
 100,
 104,
 106,
 109,
 112,
 115,
 122,
 136,
 149,
 154,
 155,
 158,
 159,
 160,
 164,
 171,
 199,
 206,
 225,
 230,
 238,
 240,
 249,
 257,
 263,
 272,
 282,
 283,
 285,
 286,
 295,
 298,
 336,
 337,
 340,
 344,
 347,
 352,
 353,
 360,
 371,
 387,
 390,
 401,
 403,
 414,
 421,
 427,
 428,
 430,
 435,
 440,
 449,
 457,
 461,
 505,
 516,
 521,
 522,
 525,
 527,
 542,
 551,
 552,
 563,
 571,
 575,
 577,
 582,
 586,
 611,
 642,
 643,
 651,
 656,
 660,
 671,
 681,
 682,
 691,
 692,
 693,
 696,
 697,
 700,
 702,
 712,
 737,
 742,
 756,
 758,
 759,
 769,
 783,
 788,
 789,
 794,
 805,
 808,
 815,
 817,
 823,
 832,
 851,
 858,
 860,
 865,
 875,
 910,
 914,
 920,
 921,
 929,
 939,
 940,
 992,
 996,
 1015,
 1018,
 1028,
 1032,
 1054,
 1081,
 1092,
 1100,
 1108,
 1110,
 1113,
 1152,
 1160,
 1171,
 1180,
 1197,
 1201,
 1206,
 1214,
 1223,
 1226,
 1227,
 1232,
 1235,
 1249,
 1283,
 1286,
 1291,
 1294,
 1304,
 1334,
 1335,
 1336,
 1341,
 1357

In [34]:
instruction, response = evaluate(get_instruction(str(9523)))
print(response)
import json
import re
json_part = re.search(r'\{(.+?)\}', response, re.DOTALL).group()
json_data = json.loads(json_part.replace(",", ''))
json_data

{
"Game price": "14.99"
}


{'Game price': '14.99'}

In [40]:
import json
import re
from tqdm import tqdm
import random
import warnings
warnings.filterwarnings("ignore")

random.shuffle(valid_user_list)

# result_save_path = f'./results/LLaMA2/{dataset_name}/'
# if not os.path.exists(result_save_path):
#     os.makedirs(result_save_path)

count = 0

pbar = tqdm(valid_user_list[:1000])
length = 0
for id in pbar:
    length += 1
    if length == 100:
        break
    df_inter_user = df_inter[df_inter['user_id:token'] == int(id)]
    his_id_list = df_inter_user['product_id:token'].tolist()[-10:]
    uid_series = dataset.token2id(dataset.uid_field, [str(id)])
    topk_score, topk_iid_list = full_sort_topk(uid_series, rec_model, test_data, k=1, device=config['device'])
    rec_list = dataset.id2token(dataset.iid_field, topk_iid_list.cpu()) 
    # next_item_info
    next_item = df_item[df_item["id:token"] == int(rec_list)]
    next_item_title = next_item["app_name:token"].values[0]
    next_item_tag = next_item["genres:token_seq"].values[0]
    next_item_price = next_item["price:float"].values[0]
    # if next_item_price == 'Free to Play':
    #     next_item_price = 0
    try:
        instruction, response = evaluate(get_instruction_steam(str(id)))
        json_part = re.search(r'\{(.+?)\}', response, re.DOTALL).group()
        json_data = json.loads(json_part.replace(",", '').replace('$', ''))
        # print(json_data)
        price = json_data['Game price']
        if 'Free' in price or 'free' in price:
            price = "Free to Play"
        # print('predict: ', price, type(price))
        # print('Label  : ', next_item_price, type(next_item_price))
        # print(price == next_item_price)
        if price == next_item_price:
            count += 1
        pbar.set_postfix({'price_acc': count / length})
    except KeyboardInterrupt:
        print('Interrupted by user')
        break
    except Exception:
        length -= 1
        continue

 10%|▉         | 99/1000 [00:40<06:09,  2.44it/s, price_acc=0.232]
